In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os


downloads_folder = os.path.expanduser('~/Downloads')

# Specify the name of your Excel file
excel_file_name = 'training_retail_supply_chain_data.xlsx' 

# Construct the full path to the Excel file
file_path = os.path.join(downloads_folder, excel_file_name)

df = pd.read_excel(file_path)

In [1]:
df.head()

NameError: name 'df' is not defined

In [ ]:

## checking the shape of the data frame
print(f'no.of rows: {df.shape[0]}')
print(f'no.of columns: {df.shape[1]}')


In [ ]:
## check for null values
df.isnull().sum()


In [ ]:
# checking the statistics of the data
df.describe()

In [ ]:
df.columns

In [ ]:
# ploting the outliers for the numeric columns using boxplot
cols = ['Units_Sold','Revenue', 'Stock_Level', 'Reorder_Threshold', 
       'Supplier_Rating', 'Discount_Applied']
for i in cols:
  plt.figure(figsize=(2,2))
  sns.boxplot(df[i])

In [ ]:
# load the above outliers into other data frame, considering only upper cutoff since no outliers in lower cutoff region
# lower_cutoff_R = df['Revenue'].quantile(0.05)
upper_cutoff_R = df['Revenue'].quantile(0.95)
# lower_cutoff_U = df['Units_Sold'].quantile(0.05)
upper_cutoff_U = df['Units_Sold'].quantile(0.95)

df_outliers = df[(df['Revenue'] > upper_cutoff_R) | (df['Units_Sold'] > upper_cutoff_U)]


In [ ]:
df_outliers.head()


In [ ]:
# Output the outliers into excel file
import os


downloads_folder = os.path.expanduser('~/Downloads')
excel_file_path = os.path.join(downloads_folder, 'Outliers_Output_file.xlsx')
os.makedirs(downloads_folder, exist_ok=True )

        # Export DataFrame to Excel
df_outliers.to_excel(excel_file_path, index=False)

In [ ]:
# plot a heat map for correlation amoung the numeric columns
cols = ['Units_Sold','Revenue', 'Stock_Level', 'Reorder_Threshold', 
       'Supplier_Rating', 'Discount_Applied']
plt.figure(figsize=(10,10))
sns.heatmap(df[cols].corr(), annot=True)

In [ ]:
## target encode category column, consider the target as revenue
df['Category'] = df.groupby('Category')['Revenue'].transform('mean')
df['Category'].head()

In [ ]:
# splitting the data
from sklearn.model_selection import train_test_split
X = df[['Units_Sold','Category','Stock_Level']]
y = df['Revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# normalize the dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# converting the normalised data to dataframes
X_train = pd.DataFrame(X_train, columns=['Units_Sold','Category','Stock_Level'])
X_test = pd.DataFrame(X_test, columns=['Units_Sold','Category','Stock_Level'])

In [ ]:
# import the models Linear Regression, Decision Tree, Random Forest
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# import the metrics mean square error and r2 score
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# creating the models for each regressor
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor()
} 

In [ ]:
# predict the y values and calculate the metrics for each model and store the metrics in a dictionary
metrics = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'{name}: MSE = {mse:.2f}, R2 = {r2:.2f}')
    metrics[name] = {'MSE': mse, 'R2': r2}   

In [ ]:
## Outputting the Regression metrics to txt file
new_filename = 'Output_Regression_metrics.txt'
data_validation_file_path = os.path.join(downloads_folder, new_filename)

with open(data_validation_file_path, 'a') as f:
    f.write(f"\n Report for the Build:\n ") 
    f.write(f"Regression Metrics :{metrics}\n ")
    f.close()

In [ ]:
########################  Classification #################################

In [ ]:
# splitting the data
from sklearn.model_selection import train_test_split
X = df[['Units_Sold','Category','Stock_Level']]
y = df['Return_Flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# normalize the dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# converting the normalised data to dataframes
X_train = pd.DataFrame(X_train, columns=['Units_Sold','Category','Stock_Level'])
X_test = pd.DataFrame(X_test, columns=['Units_Sold','Category','Stock_Level'])

In [ ]:
# import the models Logistic Regression, Naive Bayes Classification
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
# import the metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# creating the models for each classifier
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB()
}

In [ ]:
# predict the y values and calculate the metrics for each model and store the metrics in a dictionary
import warnings
warnings.filterwarnings("ignore")
metrics = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f'{name}: Accuracy = {accuracy:.2f}, Precision = {precision:.2f}, Recall = {recall:.2f}, F1 = {f1:.2}')
    metrics[name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1}

In [ ]:
## Outputting the Classification metrics to txt file
new_filename = 'Output_Classification_metrics.txt'
data_validation_file_path = os.path.join(downloads_folder, new_filename)

with open(data_validation_file_path, 'a') as f:
    f.write(f"\n Report for the Build:\n ") 
    f.write(f"Regression Metrics :{metrics}\n ")
    f.close()